In [32]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
print(application_df.shape)
application_df.head()

(34299, 12)


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [33]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=["EIN", "NAME"], inplace=True)
print(application_df.shape)
application_df.head()

(34299, 10)


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [34]:
# Determine the number of unique values in each column.
application_df.nunique().sort_values()

STATUS                       2
SPECIAL_CONSIDERATIONS       2
IS_SUCCESSFUL                2
ORGANIZATION                 4
USE_CASE                     5
AFFILIATION                  6
INCOME_AMT                   9
APPLICATION_TYPE            17
CLASSIFICATION              71
ASK_AMT                   8747
dtype: int64

In [ ]:
#For columns that have more than 10 unique values, determine the number of data points for each unique value
unique_values = ['APPLICATION_TYPE', 'CLASSIFICATION', 'ASK_AMT']
for column in unique_values:
  value_count = application_df[column].value_counts()
  for value, count in value_count.items():
      print(value)
      print(count)

In [36]:
# Look at APPLICATION_TYPE value counts for binning
application_value = application_df['APPLICATION_TYPE'].value_counts()
print (application_value)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [37]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace=list(application_value[application_value<100].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: count, dtype: int64

In [38]:
# Look at CLASSIFICATION value counts for binning
application_value = application_df['CLASSIFICATION'].value_counts()
application_value.head(50)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
C2190        1
C4200        1
C2600        1
C5200        1
C1370        1
Name: count, dtype: int64

In [39]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification = application_value.loc[application_value >1]
classification

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [40]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace=list(application_value[application_value<2].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
Other       26
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1238       10
C1278       10
C1237        9
C1235        9
C7210        7
C1720        6
C2400        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C1267        2
C1256        2
C1234        2
C1246        2
C3200        2
Name: count, dtype: int64

In [41]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, dtype=float)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [42]:
# Split our preprocessed data into our features and target arrays
target = application_df['IS_SUCCESSFUL'].values
features = application_df.drop('IS_SUCCESSFUL', axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(features,target, random_state=42)

In [43]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [44]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
features_numbers = X_train.shape[1]
hnl1=5
hnl2=10
hnl3=15

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hnl1,input_dim=features_numbers,activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hnl2, activation = 'relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hnl3, activation = 'relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_25 (Dense)            (None, 5)                 425       
                                                                 
 dense_26 (Dense)            (None, 10)                60        
                                                                 
 dense_27 (Dense)            (None, 15)                165       
                                                                 
 dense_28 (Dense)            (None, 1)                 16        
                                                                 
Total params: 666 (2.60 KB)
Trainable params: 666 (2.60 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [45]:
# Compile the model
nn.compile(loss= 'binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [46]:
# Train the model
final_model = nn.fit(X_train_scaled, y_train, validation_split=0.15, epochs=50)

Epoch 1/50
684/684 [==============================] - 3s 3ms/step - loss: 0.6234 - accuracy: 0.6783 - val_loss: 0.5819 - val_accuracy: 0.7144
Epoch 2/50
684/684 [==============================] - 2s 3ms/step - loss: 0.5787 - accuracy: 0.7143 - val_loss: 0.5669 - val_accuracy: 0.7334
Epoch 3/50
684/684 [==============================] - 2s 3ms/step - loss: 0.5657 - accuracy: 0.7240 - val_loss: 0.5572 - val_accuracy: 0.7334
Epoch 4/50
684/684 [==============================] - 2s 2ms/step - loss: 0.5599 - accuracy: 0.7262 - val_loss: 0.5573 - val_accuracy: 0.7334
Epoch 5/50
684/684 [==============================] - 2s 2ms/step - loss: 0.5577 - accuracy: 0.7270 - val_loss: 0.5563 - val_accuracy: 0.7300
Epoch 6/50
684/684 [==============================] - 2s 2ms/step - loss: 0.5564 - accuracy: 0.7280 - val_loss: 0.5563 - val_accuracy: 0.7344
Epoch 7/50
684/684 [==============================] - 2s 2ms/step - loss: 0.5560 - accuracy: 0.7286 - val_loss: 0.5567 - val_accuracy: 0.7346
Epoch 

In [47]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5555 - accuracy: 0.7300 - 347ms/epoch - 1ms/step
Loss: 0.5554905533790588, Accuracy: 0.7300291657447815


In [48]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
features_numbers = X_train.shape[1]
hnl1=10
hnl2=15
hnl3=20

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hnl1,input_dim=features_numbers,activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hnl2, activation = 'relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hnl3, activation = 'relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_29 (Dense)            (None, 10)                850       
                                                                 
 dense_30 (Dense)            (None, 15)                165       
                                                                 
 dense_31 (Dense)            (None, 20)                320       
                                                                 
 dense_32 (Dense)            (None, 1)                 21        
                                                                 
Total params: 1356 (5.30 KB)
Trainable params: 1356 (5.30 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [49]:
# Compile the model
nn.compile(loss= 'binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [50]:
# Train the model
final_model = nn.fit(X_train_scaled, y_train, validation_split=0.15, epochs=30)

Epoch 1/30
684/684 [==============================] - 3s 2ms/step - loss: 0.6050 - accuracy: 0.6867 - val_loss: 0.5654 - val_accuracy: 0.7328
Epoch 2/30
684/684 [==============================] - 2s 3ms/step - loss: 0.5608 - accuracy: 0.7239 - val_loss: 0.5538 - val_accuracy: 0.7341
Epoch 3/30
684/684 [==============================] - 2s 4ms/step - loss: 0.5548 - accuracy: 0.7294 - val_loss: 0.5493 - val_accuracy: 0.7349
Epoch 4/30
684/684 [==============================] - 2s 3ms/step - loss: 0.5525 - accuracy: 0.7292 - val_loss: 0.5492 - val_accuracy: 0.7339
Epoch 5/30
684/684 [==============================] - 2s 3ms/step - loss: 0.5499 - accuracy: 0.7308 - val_loss: 0.5494 - val_accuracy: 0.7331
Epoch 6/30
684/684 [==============================] - 1s 2ms/step - loss: 0.5489 - accuracy: 0.7310 - val_loss: 0.5535 - val_accuracy: 0.7339
Epoch 7/30
684/684 [==============================] - 1s 2ms/step - loss: 0.5483 - accuracy: 0.7318 - val_loss: 0.5484 - val_accuracy: 0.7336
Epoch 

In [51]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5568 - accuracy: 0.7250 - 395ms/epoch - 1ms/step
Loss: 0.5567846298217773, Accuracy: 0.7250145673751831


In [52]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
features_numbers = X_train.shape[1]
hnl1=20
hnl2=30
hnl3=40
hnl4=30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hnl1,input_dim=features_numbers,activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hnl2, activation = 'relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hnl3, activation = 'relu'))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hnl4, activation = 'relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_33 (Dense)            (None, 20)                1700      
                                                                 
 dense_34 (Dense)            (None, 30)                630       
                                                                 
 dense_35 (Dense)            (None, 40)                1240      
                                                                 
 dense_36 (Dense)            (None, 30)                1230      
                                                                 
 dense_37 (Dense)            (None, 1)                 31        
                                                                 
Total params: 4831 (18.87 KB)
Trainable params: 4831 (18.87 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [53]:
# Compile the model
nn.compile(loss= 'binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [54]:
# Train the model
final_model = nn.fit(X_train_scaled, y_train, validation_split=0.15, epochs=100)

Epoch 1/100
684/684 [==============================] - 3s 3ms/step - loss: 0.5814 - accuracy: 0.7137 - val_loss: 0.5533 - val_accuracy: 0.7380
Epoch 2/100
684/684 [==============================] - 2s 2ms/step - loss: 0.5557 - accuracy: 0.7276 - val_loss: 0.5476 - val_accuracy: 0.7380
Epoch 3/100
684/684 [==============================] - 2s 4ms/step - loss: 0.5517 - accuracy: 0.7298 - val_loss: 0.5508 - val_accuracy: 0.7365
Epoch 4/100
684/684 [==============================] - 2s 3ms/step - loss: 0.5497 - accuracy: 0.7320 - val_loss: 0.5568 - val_accuracy: 0.7352
Epoch 5/100
684/684 [==============================] - 2s 3ms/step - loss: 0.5487 - accuracy: 0.7330 - val_loss: 0.5465 - val_accuracy: 0.7383
Epoch 6/100
684/684 [==============================] - 2s 3ms/step - loss: 0.5475 - accuracy: 0.7326 - val_loss: 0.5518 - val_accuracy: 0.7359
Epoch 7/100
684/684 [==============================] - 2s 2ms/step - loss: 0.5466 - accuracy: 0.7331 - val_loss: 0.5491 - val_accuracy: 0.7370

In [55]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5762 - accuracy: 0.7275 - 369ms/epoch - 1ms/step
Loss: 0.5761537551879883, Accuracy: 0.7274635434150696


In [57]:
# Export our model to HDF5 file
from google.colab import files
nn.save ('/Users/semanur/Desktop/Module21/deep-learning-challenge/Final_deep_learning.h5')
files.download('/Users/semanur/Desktop/Module21/deep-learning-challenge/Final_deep_learning.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>